Install neo4j graphdatascience client ([Documentation](https://neo4j.com/docs/graph-data-science/current/)) and openai

In [ ]:
%%capture
!pip install graphdatascience 
!pip install openai  
!pip install retry

Import our usual suspects

In [ ]:
import pandas as pd
from graphdatascience import GraphDataScience 
import os
import openai
from retry import retry

Register for a sandbox and create an empty sandbox  https://sandbox.neo4j.com

In [ ]:
# Capture connection string and auth info
connectionUrl = input("Neo4j Database Url: ")
username = 'neo4j' #input("User name: ")
password = input("Password: ")
# Other env variables
os.environ["OPENAI_API_KEY"] = 'PASTE YOUR API KEY HERE'
openai.api_key = os.getenv('OPENAI_API_KEY')

Neo4j Database Url: bolt://44.200.225.7:7687
Password: document-authorization-coders


In [ ]:
gds = GraphDataScience(connectionUrl, auth=(username, password))
gds.set_database('neo4j')
print(gds.version())


2.3.5


In [ ]:
transaction_df = pd.DataFrame([
    {'name': 'Tom', 'merchant':'Amazon', 'amount': 100},
    {'name': 'Tom', 'merchant':'Dustin', 'amount': 50499},
    {'name': 'Tom', 'merchant':'eBay', 'amount': 220},
    {'name': 'Stefan', 'merchant':'Amazon', 'amount': 220},
    {'name': 'Stefan', 'merchant':'Dustin', 'amount': 399},
    {'name': 'Stefan', 'merchant':'eBay', 'amount': 1499},
    {'name': 'Stefan', 'merchant':'Bikes.de', 'amount': 22000},
    {'name': 'Kristof', 'merchant':'Amazon', 'amount': 423},
    {'name': 'Kristof', 'merchant':'Dustin', 'amount': 530},
    {'name': 'Kristof', 'merchant':'Hello Fresh', 'amount': 1050},
    {'name': 'Kristof', 'merchant':'Steam', 'amount': 230},
    {'name': 'Kristof', 'merchant':'Activision', 'amount': 783},
    {'name': 'Håkan', 'merchant':'Hello Fresh', 'amount': 2100},
    {'name': 'Håkan', 'merchant':'Steam', 'amount': 230},
    {'name': 'Håkan', 'merchant':'Activision', 'amount': 783},
    
], columns = ['name', 'merchant', 'amount'])
transaction_df.head(15)

,name,merchant,amount
0,Tom,Amazon,100
1,Tom,Dustin,50499
2,Tom,eBay,220
3,Stefan,Amazon,220
4,Stefan,Dustin,399
5,Stefan,eBay,1499
6,Stefan,Bikes.de,22000
7,Kristof,Amazon,423
8,Kristof,Dustin,530
9,Kristof,Hello Fresh,1050


In [ ]:
# This is really not required for this small sample
gds.run_cypher("create constraint if not exists for (p:Person) require (p.name) is node key")
gds.run_cypher("create constraint if not exists for (p:Merchant) require (p.name) is node key")


""


In [ ]:

# Create a graph for (:Person)-[:transacted_with]->(:Merchant)
gds.run_cypher(
    """
    unwind $transactions as transaction
    merge (p:Person{name: transaction['name']})
    merge (m:Merchant{name: transaction['merchant']})
    merge (p)-[tx:TRANSACTED_WITH]->(m)
       set tx.amount = transaction['amount']
    """,
    params = { 'transactions': transaction_df.to_dict(orient='records') }
)

""


Let's get this party started

In [ ]:
G, res = gds.graph.project(
    "shopping",                                     #  Graph name
    ["Person", "Merchant"],                         #  Node projection
    {"TRANSACTED_WITH": {"properties": "amount", "orientation": "REVERSE"}}   #  Relationship projection
)


Loading:   0%|          | 0/100 [00:00<?, ?%/s]

In [ ]:
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")


Graph 'shopping' node count: 11
Graph 'shopping' node labels: ['Merchant', 'Person']


In [ ]:
write_results = gds.nodeSimilarity.write(
    G,
    writeRelationshipType= 'SIMILAR_CUSTOMERS' ,
    writeProperty= 'SIM_SCORE',
    relationshipWeightProperty= 'amount'
)

write_results

preProcessingMillis                                                       8
computeMillis                                                           104
writeMillis                                                             377
postProcessingMillis                                                     -1
nodesCompared                                                             7
relationshipsWritten                                                     30
similarityDistribution    {'p1': 0.004452347755432129, 'max': 0.49714466...
configuration             {'topK': 10, 'writeConcurrency': 4, 'similarit...
Name: 0, dtype: object

In [ ]:
# Remove symmetric SIMILAR_CUSTOMERS relationships
gds.run_cypher("""
    MATCH (m:Merchant)-[r:SIMILAR_CUSTOMERS]->(n:Merchant) WHERE (n)-[:SIMILAR_CUSTOMERS]->(m) AND id(m)<id(n)
    DELETE r
""")


""


In [ ]:
merchant_df = pd.DataFrame([
    {'merchant':'Amazon', 'description': 'The worlds largest online retailer, offers a vast array of products ranging from books and electronics to clothing and groceries. With its convenient one-stop-shop approach, Amazon has revolutionized the way people shop, making it possible to buy almost anything online and have it delivered right to their doorstep'},
    {'merchant':'Dustin', 'description': 'A leading Nordic reseller of IT products and services, provides businesses and consumers with high-quality computer hardware, software, and accessories. With a broad range of products and a focus on customer satisfaction, Dustin is a trusted partner for all your IT needs.'},
    {'merchant':'eBay', 'description': 'The popular online auction site, allows buyers and sellers to trade goods and services in a global marketplace. From vintage collectibles to the latest gadgets, eBay has something for everyone. With millions of listings and a reputation for reliability, eBay is a great place to buy and sell almost anything.'},   
    {'merchant':'Bikes.de', 'description': 'A premier cycling retailer in Germany offering a wide selection of bicycles, parts, and accessories for enthusiasts and casual riders alike. With a focus on quality and performance, Bikes.de has everything you need to hit the road or the trails in style.'},
    {'merchant':'Hello Fresh', 'description': 'The meal kit delivery service, makes it easy to cook delicious and healthy meals at home. With pre-portioned ingredients and easy-to-follow recipes, Hello Fresh takes the hassle out of meal planning and prep, so you can spend more time enjoying your food and less time in the kitchen.'},
    {'merchant':'Steam', 'description': 'The popular gaming platform, offers a massive library of games for PC, Mac, and Linux. With features like cloud saves, automatic updates, and social networking, Steam makes it easy to find, buy, and play your favorite games with friends and fellow gamers from around the world.'},
    {'merchant':'Activision', 'description': 'One of the largest video game publishers, is responsible for some of the biggest and most successful game franchises of all time. From Call of Duty to World of Warcraft, their games are known for immersive gameplay, stunning graphics, and engaging storylines. With a commitment to innovation and excellence, Activision continues to push the boundaries of gaming and entertainment.'},
], columns = ['merchant', 'description'])
pd.set_option('max_colwidth', None)
merchant_df.head(10)

In [ ]:
merchant_df.to_csv('merchant_df')

In [ ]:
#Or load the CSV file here - get the csv from Github page

# merchant_df = pd.read_csv('/content/merchant_df')

In [ ]:
def get_embedding(text: str, model="text-embedding-ada-002") -> list[float]:
    return openai.Embedding.create(input=[text], model=model)["data"][0]["embedding"]


In [ ]:
merchant_df['embedding'] = merchant_df['description'].apply(get_embedding)
merchant_df.head(10)

,merchant,description,embedding
0,Amazon,"The worlds largest online retailer, offers a vast array of products ranging from books and electronics to clothing and groceries. With its convenient one-stop-shop approach, Amazon has revolutionized the way people shop, making it possible to buy almost anything online and have it delivered right to their doorstep","[0.011217989027500153, -0.023190975189208984, 0.004891748074442148, -0.010267951525747776, -0.013917101547122002, 0.012495191767811775, -0.013325687497854233, -0.009028499014675617, 0.0145210986956954, -0.02868986502289772, -0.005454849451780319, -0.0064080324955284595, -0.013564770109951496, -0.021165067330002785, 0.012136568315327168, 0.010884532704949379, 0.02906736359000206, -0.0117527786642313, -0.030300524085760117, -0.03017469309270382, -0.03470467031002045, 0.006433199159801006, -0.008588084951043129, 0.0018167102243751287, 0.0008493710192851722, 0.014395265839993954, 0.012470024637877941, -0.01804441586136818, 0.012501482851803303, -0.017843082547187805, -0.008235753513872623, -0.023027392104268074, -0.03354701027274132, -0.01327535416930914, -0.00792746338993311, 0.025141382589936256, -0.0002974764793179929, -0.011098448187112808, -0.004479645751416683, -0.027431538328528404, 0.019189493730664253, 0.024600300937891006, -0.003743524197489023, -0.0026487791910767555, -0.026173211634159088, 0.001267764950171113, -0.004325500689446926, -0.03790082409977913, -0.01585492677986622, 0.018056998029351234, 0.018082166090607643, 0.03873131796717644, 0.0007911733700893819, -0.001964563736692071, -0.0015272949822247028, 0.007638047449290752, -0.01321243867278099, 0.005649890284985304, 0.022649893537163734, -0.013325687497854233, -0.005829202011227608, 0.02249889448285103, -0.04303479939699173, -0.01277831569314003, -0.011771652847528458, -0.00010322217713110149, -0.03961214795708656, 0.005395079031586647, 0.010098077356815338, 0.008776834234595299, 0.02725537307560444, 0.015691343694925308, 0.01468468178063631, 0.022914143279194832, 0.010267951525747776, 0.007537381257861853, -0.006933384109288454, 0.029646195471286774, 0.015653593465685844, 0.007562547922134399, 0.012451150454580784, -0.017679501324892044, 0.02098890207707882, -0.022524060681462288, 0.008235753513872623, 0.0046180617064237595, -0.012727982364594936, 0.02449963614344597, -0.014835680834949017, 0.007285716012120247, 0.011557737365365028, -0.02558179758489132, 0.004042376764118671, 0.007147300057113171, -0.016333090141415596, 0.009380831383168697, -0.018547747284173965, 0.030728356912732124, 0.003321984549984336, -0.03742265701293945, ...]"
1,Dustin,"A leading Nordic reseller of IT products and services, provides businesses and consumers with high-quality computer hardware, software, and accessories. With a broad range of products and a focus on customer satisfaction, Dustin is a trusted partner for all your IT needs.","[-0.008895962499082088, -0.018266556784510612, 0.008278940804302692, -0.019636210054159164, -0.01093009952455759, 0.029264461249113083, -0.04358479008078575, 0.0018307239515706897, 0.014849205501377583, -0.015988321974873543, 0.0016290053026750684, 0.03257332369685173, -0.0014315244043245912, -0.0061973393894732, -0.008285720832645893, 0.017195243388414383, 0.016259539872407913, -0.025575891137123108, 0.004403908271342516, -0.023270536214113235, -0.041496410965919495, -0.007411041762679815, 0.023568876087665558, -0.000618716876488179, 0.00010244849545415491, -0.006027827970683575, 0.00817723385989666, -0.012842189520597458, 0.00825181882828474, 0.0034122660290449858, 0.010001176968216896, -0.009831666015088558, -0.0014781400095671415, -0.02506057731807232, -0.01562217716127634, 0.006682142149657011, -0.008828157559037209, -0.0029325485229492188, 0.00724153034389019, -0.017222365364432335, 0.015445885248482227, -0.010204590857028961, 0.00563117116689682, -0.0058311945758759975, 0.007675479631870985, 0.022456880658864975, -0.014333890751004219, 0.0003104178758803755, 0.0006966921500861645, -0.01376

In [ ]:
gds.run_cypher("""
  unwind $merchant_des as merchant_des
  MATCH  (m:Merchant {name: merchant_des['merchant']})
  set m.embedding = merchant_des['embedding'],
      m.description=merchant_des['description']""",
  params = { 'merchant_des': merchant_df.to_dict(orient='records') }
)



""


In [ ]:
G2, result_2 = gds.graph.project(
  "embedG3",                                
  {                                                     
                            
    "Merchant": {"properties": {"embedding": {"defaultValue": [0.0]}}}    
  },
  {"SIMILAR_CUSTOMERS": {"properties": "SIM_SCORE", "orientation": "UNDIRECTED"} }                                    
  )

In [ ]:
fastrp_res = gds.fastRP.write(
    G2,                                #  Graph object
    featureProperties=["embedding"],   #  Configuration parameters
    embeddingDimension=128,
    propertyRatio=0.5,
    iterationWeights=[0, 0, 1.0, 1.0],
    normalizationStrength=0.05,
    writeProperty="Node_Embedding"
)

FastRP:   0%|          | 0/100 [00:00<?, ?%/s]

In [ ]:
G.drop()
G2.drop()
gds.run_cypher("""
  MATCH (n)
  DETACH DELETE n"""
)
